In [4]:
# @title
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

#####################
# MONTAGEM DO DRIVE #
#####################
from google.colab import drive
if not os.path.ismount('/content/drive'):
    print("Montando o Google Drive...")
    drive.mount('/content/drive')

###########################
# CARREGAMENTO DE MÓDULOS #
###########################

# Caminho dos módulos do projeto armazenados no Google Drive
caminho_modulos = '/content/drive/MyDrive/Colab Notebooks/VC/Filtragem/modulos'
from pydoc import importfile
# Importa um módulo Python utilizando o caminho de um arquivo Python
filtros = importfile(caminho_modulos+'/filtros.py')

########################################################################
# CAMINHO DAS IMAGENS E VARIAVEL GLOBAL PARA A IMAGEM CORRENTE (ATUAL) #
########################################################################

# Caminho das imagens do projeto armazenadas no Google Drive
caminho_imagens = '/content/drive/MyDrive/Colab Notebooks/VC/Filtragem/imagens'

# Variáveis globais para armazenar as imagens original e a corrente
imagem_original = None
imagem_corrente = None


##################################################################
# FUNÇÕES CALLBACK ASSOCIADAS AOS ITENS DA INTERFACE DO  USUÁRIO #
##################################################################

# Função para carregar e exibir a imagem selecionada
def abrir_imagem(b):
    output_image.clear_output()  # Limpar qualquer saída anterior
    caminho_arquivo = os.path.join(caminho_imagens, dropdown_arquivos.value)
    global imagem_original
    global imagem_corrente
    imagem_original = filtros.read_image(caminho_arquivo)
    imagem_corrente = imagem_original
    if imagem_corrente is not None:
        exibir_imagem(imagem_corrente)
    else:
        print('Erro ao carregar a imagem.')

# Função para fechar a imagem corrente
def fechar_imagem(b):
    output_image.clear_output()
    global imagem_corrente
    imagem_corrente = None

# Função para redimensionar a imagem corrente
def redimensionar(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.resize_image(imagem_corrente,
                                             redimensionar_largura.value,
                                             redimensionar_altura.value)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para converter para escala de cinza a imagem corrente
def escala_cinza(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.grayscale_image(imagem_corrente)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o filtro media na imagem corrente
def filtro_media(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.average_filter(imagem_corrente, slider_media.value)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o filtro gaussiano na imagem corrente
def filtro_gaussiano(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.gaussian_filter(imagem_corrente, slider_gaussiano.value)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o filtro mediana na imagem corrente
def filtro_mediana(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.median_filter(imagem_corrente, slider_mediana.value)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o filtro sobel na imagem corrente
def filtro_sobel(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.sobel_filter(imagem_corrente)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o filtro laplaciano na imagem corrente
def filtro_laplaciano(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.laplacian_filter(imagem_corrente)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')


# Função para aplicar o filtro highboost na imagem corrente
def filtro_highboost(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.highboost_filter(imagem_corrente, slider_highboost.value)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

# Função para aplicar o ruído sal e pimenta na imagem corrente
def ruido_salpimenta(b):
    global imagem_corrente
    if imagem_corrente is not None:
        imagem_corrente = filtros.salt_and_pepper_noise(imagem_corrente)
        exibir_imagem(imagem_corrente)
    else:
        print('Não existe imagem carregada.')

###########################################
# FUNÇÕES AUXILIARES ÀS FUNÇÕES CALLBACKS #
###########################################

# Função para listar arquivos de imagem no diretório atual
def listar_arquivos(diretorio):
    arquivos = os.listdir(diretorio)
    arquivos_img = [arquivo for arquivo in arquivos if arquivo.lower().endswith(('png', 'jpg', 'jpeg'))]
    return arquivos_img

# Função para exibir uma imagem utilizando a matplotlib
def exibir_imagem(image):
    with output_image: # Widget para a interface gráfica web
        output_image.clear_output(wait=True)  # Limpar a saída anterior, mas espera até o novo conteúdo estar pronto
        dpi = 100  # Define a densidade de pixels por polegada
        height, width = image.shape[:2]  # Obtém a altura e a largura da imagem
        # a imagem deve ser exibida em seu tamanho real sem redimensionamento
        fig, ax = plt.subplots(figsize=(width / dpi, height / dpi), dpi=dpi)
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        plt.show()

def restaurar_imagem(b):
    if imagem_original is not None:
        global imagem_corrente
        imagem_corrente = imagem_original
        exibir_imagem(imagem_corrente)  # Restaura a imagem original

#################################
# ITENS DA INTERFACE DO USUÁRIO #
#################################

# Header
header = widgets.HTML('<font color="white" face="sans-serif"><center><h1>Filtragem de Imagens</h1></center></font>',
                      layout=widgets.Layout(height='auto'))

# Botão para restaurar a imagem (será utilizado em várias abas)
button_restaurar = widgets.Button(description='Restaurar')
# Função atribuída ao botão restaurar
button_restaurar.on_click(restaurar_imagem)

###################
# ABAS PRINCIPAIS #
###################

# Criar abas
tab = widgets.Tab(
    layout=widgets.Layout(min_width='600px', min_height='50px')  # Definir tamanho mínimo
)

# Definir os títulos de cada aba
tab.set_title(0, 'ARQUIVO')
tab.set_title(1, 'IMAGEM')
tab.set_title(2, 'FILTROS')

# Output widget para exibir uma imagem na interface
output_image = widgets.Output(layout={'border': '1px solid black'})

# GridBox para organizar o cabeçalho, as abas e a imagem de saída
grid = widgets.GridBox(children=[header, tab, output_image],
               layout=widgets.Layout(
                   grid_template_rows='auto auto auto',
                   grid_template_columns='1fr',  #coluna flexível para ocupar o espaço disponível.
                   justify_items='center',
                   align_items='center',
                   grid_gap='10px'
               ))

# Contêiner pai para centralizar o GridBox
container = widgets.Box([grid], layout=widgets.Layout(display='flex', justify_content='center', align_items='center'))

# Exibir o contêiner pai
display(container)


########################
# CONTEUDO ABA ARQUIVO #
########################

# Dropdown para selecionar um arquivo de imagem
dropdown_arquivos = widgets.Dropdown(
    options=listar_arquivos(caminho_imagens),
    description='Imagem:',
)
# Botões
button_abrir = widgets.Button(description='Abrir')
button_fechar = widgets.Button(description='Fechar')
# Atribuir funções aos botões
button_abrir.on_click(abrir_imagem)
button_fechar.on_click(fechar_imagem)

########################
# CONTEUDO ABA IMAGEM  #
########################

########################
# ITENS REDIMENSIONAR  #
########################

# Botão
button_redimensionar = widgets.Button(description='Aplicar')
# Atribuir função aos botão
button_redimensionar.on_click(redimensionar)
# Caixa de texto para redimensionar imagem
redimensionar_largura = widgets.BoundedIntText(value=50,
                                      min=0,
                                      max=200,
                                      step=1,
                                      description='Largura (%):',
                                      disabled=False,
                                      layout=widgets.Layout(width='150px')
                                  )
redimensionar_altura = widgets.BoundedIntText(value=50,
                                      min=0,
                                      max=200,
                                      step=1,
                                      description='Altura (%):',
                                      disabled=False,
                                     layout=widgets.Layout(width='150px')
                                  )

# Box para os itens de Redimensionar
box_redimensionar = widgets.Box([redimensionar_largura, redimensionar_altura, button_redimensionar, button_restaurar],
                                layout=widgets.Layout(justify_content='center',gap='1px',margin='0px'))

##########################
# ITENS ESCALA DE CINZA  #
##########################

# Botão
button_escala_cinza = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_escala_cinza.on_click(escala_cinza)
# Box para os itens de Escala de Cinza
box_escala_cinza = widgets.Box([button_escala_cinza, button_restaurar],
                               layout=widgets.Layout(justify_content='center',margin='0px'))

# Criar subabas para a aba Imagem
subtab_imagem = widgets.Tab(
    children=[
        widgets.VBox([box_redimensionar]),
        widgets.VBox([box_escala_cinza])
    ],
    layout=widgets.Layout(min_width='500px', min_height='40px')  # Definir tamanho mínimo
)
subtab_imagem.set_title(0, 'Redimensionar')
subtab_imagem.set_title(1, 'Escala de Cinza')


#########################
# CONTEUDO ABA FILTROS  #
#########################

#######################
#    ITENS MEDIA       #
#######################

# Slider
slider_media = widgets.IntSlider(value=3, min=3, max=11, step=2)
# Botão
button_media = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_media.on_click(filtro_media)


#######################
#    ITENS GAUSSIANO  #
#######################

# Slider
slider_gaussiano = widgets.IntSlider(value=3, min=3, max=11, step=2)
# Botão
button_gaussiano = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_gaussiano.on_click(filtro_gaussiano)

#######################
#    ITENS MEDIANA  #
#######################

# Slider
slider_mediana = widgets.IntSlider(value=3, min=3, max=11, step=2)
# Botão
button_mediana = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_mediana.on_click(filtro_mediana)

#####################
# ITENS SUAVIZACAO  #
#####################

# Boxes para os itens de Suavização
box_media = widgets.Box([slider_media, button_media, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))
box_gaussiano = widgets.Box([slider_gaussiano, button_gaussiano, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))
box_mediana = widgets.Box([slider_mediana, button_mediana, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))

# Criar subabas com os boxes de Suavização
subtab_suavizacao = widgets.Tab(
    children=[
        widgets.VBox([box_media]),
        widgets.VBox([box_gaussiano]),
        widgets.VBox([box_mediana])
    ],
    layout=widgets.Layout(min_width='500px', min_height='40px')  # Definir tamanho mínimo
)
subtab_suavizacao.set_title(0, 'Média')
subtab_suavizacao.set_title(1, 'Gaussiano')
subtab_suavizacao.set_title(2, 'Mediana')

#####################
# ITENS AGUÇAMENTO  #
#####################


#######################
#    ITENS SOBEL      #
#######################

# Botão
button_sobel = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_sobel.on_click(filtro_sobel)


########################
#    ITENS LAPLACIANO  #
########################

# Botão
button_laplaciano = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_laplaciano.on_click(filtro_laplaciano)

########################
#    ITENS HIGH BOOST  #
########################

# Slider
slider_highboost = widgets.FloatSlider(value=1.5, min=1, max=3, step=0.5)
# Botão
button_highboost = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_highboost.on_click(filtro_highboost)

# Boxes para os itens de Aguçamento
box_sobel = widgets.Box([button_sobel, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))
box_laplaciano = widgets.Box([button_laplaciano, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))
box_highboost = widgets.Box([slider_highboost,button_highboost, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))

# Criar subabas com os boxes de Aguçamento
subtab_agucamento = widgets.Tab(
    children=[
        widgets.VBox([box_sobel]),
        widgets.VBox([box_laplaciano]),
        widgets.VBox([box_highboost])
    ],
    layout=widgets.Layout(min_width='500px', min_height='40px')  # Definir tamanho mínimo
)
subtab_agucamento.set_title(0, 'Sobel')
subtab_agucamento.set_title(1, 'Laplaciano')
subtab_agucamento.set_title(2, 'High Boost')

################
# ITENS RUÍDO  #
################

###########################
#    ITENS SAL E PIMENTA  #
###########################

# Botão
button_salpimenta = widgets.Button(description='Aplicar')
# Atribuir função ao botão
button_salpimenta.on_click(ruido_salpimenta)

# Box para os itens de Ruído
box_salpimenta = widgets.Box([button_salpimenta, button_restaurar], layout=widgets.Layout(justify_content='center',gap='1px'))

# Criar subabas com os box de Ruído
subtab_ruido = widgets.Tab(
    children=[
        widgets.VBox([box_salpimenta]),
    ],
    layout=widgets.Layout(min_width='500px', min_height='40px')  # Definir tamanho mínimo
)
subtab_ruido.set_title(0, 'Sal e pimenta')

# Criar subabas para a aba Filtros
subtab_filtros = widgets.Tab(
    children=[
        widgets.VBox([subtab_suavizacao]),
        widgets.VBox([subtab_agucamento]),
        widgets.VBox([subtab_ruido])
    ],
    layout=widgets.Layout(min_width='500px', min_height='40px')  # Definir tamanho mínimo
)
subtab_filtros.set_title(0, 'Suavização')
subtab_filtros.set_title(1, 'Aguçamento')
subtab_filtros.set_title(2, 'Ruído')


# Definir o conteúdo de cada subaba
tab.children = [
    widgets.HBox([dropdown_arquivos,button_abrir, button_fechar],layout=widgets.Layout(align_items='center', justify_content='center')),
    widgets.HBox([subtab_imagem],layout=widgets.Layout(align_items='center', justify_content='center')),
    widgets.HBox([subtab_filtros], layout=widgets.Layout(align_items='center', justify_content='center'))
]

################ FIM DO SCRIPT ########################

Box(children=(GridBox(children=(HTML(value='<font color="white" face="sans-serif"><center><h1>Filtragem de Ima…